In [1]:
import ROOT
import numpy as np
from IPython.display import display
import math

import histdump as hd
import gptools as gp
import figtools as fig
from fittools import Fitter, extended_logl
import pdfs

Welcome to JupyROOT 6.13/02


In [2]:
boundaries = [0., 5.]
n_bins = 5
bin_width = (boundaries[1] - boundaries[0]) / n_bins

In [3]:
rnd = ROOT.TRandom3(42)
l, r = boundaries
n_data = 50
data = [l + (r-l) * rnd.Rndm() for _ in range(n_data)]

In [4]:
myFunc = pdfs.Linear(boundaries)

In [5]:
def fcn(npar, gin, f, par, iflag):
    a = par[0]
    n = par[1]
    
    ln_f = lambda x: math.log(myFunc(x, a))
    try:
        f[0] = extended_logl(ln_f, n, data)
    except ValueError:
        f[0] = math.inf

fitter = Fitter()
fitter.add_param('a', (-3., 3.))
fitter.add_param('n', (0., 2.*n_data))
fitter.fit(fcn)

 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 a            0.00000e+00  6.00000e-04   -3.00000e+00  3.00000e+00
     2 n            5.00000e+01  1.00000e-02    0.00000e+00  1.00000e+02
 **********
 **    1 **MIGRAD
 **********
 FIRST CALL TO USER FUNCTION AT NEW START POINT, WITH IFLAG=4.
 START MIGRAD MINIMIZATION.  STRATEGY  1.  CONVERGENCE WHEN EDM .LT. 1.00e-04
 FCN=-65.1293 FROM MIGRAD    STATUS=INITIATE        8 CALLS           9 TOTAL
                     EDM= unknown      STRATEGY= 1      NO ERROR MATRIX       
  EXT PARAMETER               CURRENT GUESS       STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  a            0.00000e+00   6.00000e-04   2.00000e-04   4.88001e+01
   2  n            5.00000e+01   1.00000e-02   2.00000e-04  -7.10543e-09
 MIGRAD MINIMIZATION HAS CONVERGED.
 FCN=-66.2529 FROM MIGRAD    STATUS=CONVERGED      75 CALLS          76 TOTAL
                     EDM=3.597

In [6]:
fitter.print_values()

# 0: a = -0.10+/-0.07
# 1: n = 50+/-10


In [7]:
a = fitter.get_value(0).n
norm = bin_width * float(len(data))

In [8]:
bin_edges = np.linspace(*boundaries, n_bins+1)

hd.make_hist_file(data=data, bin_edges=bin_edges, file_name='data.csv')

f = lambda x: myFunc(x, a) * norm
hd.make_steps_file(func=f, bin_edges=bin_edges, file_name='f.csv')

cmd = gp.create('data', code=r'''
load 'csvdata.cfg'
datafile = 'data.csv'
fitfile = 'f.csv'

set yrange [0:]

plot datafile u 2:5:1:3:4:6 with xyerrorbars lc black pt 7 ps 0.7 t 'Data',\
     fitfile with steps t 'Fit'
''')
!{cmd} > /dev/null
fig.show_fig('data.png')